Escolha da task na pasta Training

In [0]:
TASK = 'assin-ptpt-rte'

Baixar o repositório do mt-dnn e requisitos

In [4]:
!git clone https://github.com/namisan/mt-dnn
%cd mt-dnn 
!git checkout 60aa9dc4ec1
!pip install pytorch_pretrained_bert==0.5.1
!pip install sentencepiece 
!pip install seqeval  
!pip install tensorboardX

Cloning into 'mt-dnn'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 1577 (delta 172), reused 244 (delta 151), pack-reused 1288
Receiving objects: 100% (1577/1577), 970.00 KiB | 4.31 MiB/s, done.
Resolving deltas: 100% (861/861), done.
/content/drive/My Drive/html/assin-ptpt-rte/mt-dnn
     |████████████████████████████████| 92kB 2.5MB/s 
     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=f7e4a8034595e06a40b3505ba1ae2b61fbe4967866f7554444048fe956662df3
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
     |████████████████████████████████| 204kB 2.8MB/s 


Baixar modelos de rede neural

In [7]:
#When mount on Google Drive on Colab, please run download.sh until it downloads mt_dnn_base_uncased.pt
#Or run !bash ../download_model.sh that it will do it for you
!bash download.sh

Archive:  xlnet_cased_large.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-24_H-1024_A-16/spiece.model  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_config.json  
roberta.base/
roberta.base/dict.txt
roberta.base/model.pt
roberta.base/NOTE
roberta.large/
roberta.large/dict.txt
roberta.large/model.pt
roberta.large/NOTE
Archive:  SciTailV1.1.zip
   creating: SciTailV1.1/
   creating: SciTailV1.1/snli_format/
  inflating: SciTailV1.1/snli_format/scitail_1.0_test.txt  
  inflating: SciTailV1.1/snli_format/scitail_1.0_train.txt  
  inflating: SciTailV1.1/snli_format/scitail_1.0_dev.txt  
  inflating: SciTailV1.1/snli_format/README.txt  
   creating: SciTailV1.1/dgem_format/
  inflating: SciTailV1.1/dgem_format/scitail_1.0_structure_dev.tsv  
  inflati

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Mover dados da task

In [8]:
!mkdir data/canonical_data

Aplicar patch em metrics.py para incluir *f1_nomean* e *mse*

In [9]:
!patch data_utils/metrics.py < ../metrics.patch

patching file data_utils/metrics.py


Mover dados de entrada

In [0]:
%mv ../move_input.sh move_input.sh
!bash move_input.sh $TASK

Preprocess data

In [19]:
!python prepro_std.py --do_lower_case --root_dir data/canonical_data --task_def {TASK}.yaml

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
03/07/2020 03:11:35 Task assin-ptpt-rte
03/07/2020 03:11:35 data/canonical_data/bert_uncased_lower/assin-ptpt-rte_train.json
03/07/2020 03:11:36 data/canonical_data/bert_uncased_lower/assin-ptpt-rte_dev.json
03/07/2020 03:11:36 data/canonical_data/bert_uncased_lower/assin-ptpt-rte_test.json


Habilitar scores dos datasets de teste

In [0]:
!patch train.py < ../train.patch

patching file train.py
Reversed (or previously applied) patch detected!  Assume -R? [n] 
Apply anyway? [n] 
Skipping patch.
1 out of 1 hunk ignored -- saving rejects to file train.py.rej


Corrigir mt_dnn/model.py e prepro_std.py

In [0]:
!patch mt_dnn/model.py < ../model.patch
!patch prepro_std.py < ../prepro_std.patch

Train task

In [0]:
!python train.py --do_lower_case --init_checkpoint mt_dnn_models/mt_dnn_base_uncased.pt --task_def {TASK}.yaml --train_datasets {TASK} --test_datasets {TASK} --tensorboard --data_dir data/canonical_data/bert_base_uncased_lower

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Namespace(adam_eps=1e-06, answer_att_hidden_size=128, answer_att_type='bilinear', answer_dropout_p=0.1, answer_mem_drop_p=0.1, answer_mem_type=1, answer_merge_opt=1, answer_num_turn=5, answer_opt=0, answer_rnn_type='gru', answer_sum_att_type='bilinear', answer_weight_norm_on=False, batch_size=8, batch_size_eval=8, bert_dropout_p=0.1, bert_l2norm=0.0, bert_model_type='bert-base-uncased', cuda=False, data_dir='data/canonical_data/bert_uncased_lower', data_sort_on=False, do_lower_case=True, dropout_p=0.1, dropout_w=0.0, dump_state_on=False, embedding_opt=0, encoder_type=<EncoderModelType.BERT: 1>, epochs=5, fp16=False, fp16_opt_level='O1', freeze_layers=-1, global_grad_clipping=1.0, glue_format_on=False, grad_accumulation_step=1, grad_clipping=0, have_lr_scheduler=True, init_checkpoint='mt_dnn_models/mt_dnn_base_uncased.pt', init_ratio=1, label_size='3', learning_rate=5e-05, log_file='mt-dnn-train.lo

Mostrar resultados do *train.py* para arquivos de teste

In [5]:
import json

filepath = 'checkpoint/{}_test_scores_4.json'.format(TASK) 
with open(filepath) as f:
    scores = json.load(f)

print(scores['metrics'])

{'Pearson': 85.01105592648992, 'MSE': 13.714753768382353}


Abrir *Tensorboard*.

Executa no *Chrome* normalmente, mas não funciona no *Firefox*

In [0]:
%load_ext tensorboard
%tensorboard  --logdir tensorboard_logdir